In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms


image_path = './'
transform = transforms.Compose([transforms.ToTensor()])

mnist_train_dataset = torchvision.datasets.MNIST(
    root=image_path, 
    train=True, 
    transform=transform, 
    download=True
)
mnist_test_dataset = torchvision.datasets.MNIST(
    root=image_path,
    train=False, 
    transform=transform, 
    download=False
)

batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)

In [7]:
mnist_train_dataset[0][0].size()

torch.Size([1, 28, 28])

In [23]:
hidden_units = [32, 16]
image_size = mnist_train_dataset[0][0].shape
print(f"Image Size: {image_size}")
input_size = image_size[0] * image_size[1] * image_size[2]
print(f"Input Size: {input_size}")

all_layers = [nn.Flatten()]
for hidden_unit in hidden_units:
    print(f"Adding Linear Layer with {input_size} inputs and {hidden_unit} outputs.")
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

print(f"Adding Output (Linear) Layer with {hidden_units[-1]} inputs and {10} outputs.")
all_layers.append(nn.Linear(hidden_units[-1], 10))
all_layers.append(nn.Softmax(dim=1))
model = nn.Sequential(*all_layers)

model

Image Size: torch.Size([1, 28, 28])
Input Size: 784
Adding Linear Layer with 784 inputs and 32 outputs.
Adding Linear Layer with 32 inputs and 16 outputs.
Adding Output (Linear) Layer with 16 inputs and 10 outputs.


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
  (6): Softmax(dim=1)
)

In [24]:
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = model.to(device)

cpu


In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

torch.manual_seed(1)
num_epochs = 20
for epoch in range(num_epochs):
    accuracy_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_dl.dataset)
    print(f'Epoch {epoch}  Accuracy {accuracy_hist_train:.4f}')

Epoch 0  Accuracy 0.8253
Epoch 1  Accuracy 0.9164
Epoch 2  Accuracy 0.9284
Epoch 3  Accuracy 0.9367
Epoch 4  Accuracy 0.9422
Epoch 5  Accuracy 0.9461
Epoch 6  Accuracy 0.9510
Epoch 7  Accuracy 0.9534
Epoch 8  Accuracy 0.9567
Epoch 9  Accuracy 0.9588
Epoch 10  Accuracy 0.9608
Epoch 11  Accuracy 0.9624
Epoch 12  Accuracy 0.9637
Epoch 13  Accuracy 0.9656
Epoch 14  Accuracy 0.9662
Epoch 15  Accuracy 0.9679
Epoch 16  Accuracy 0.9680
Epoch 17  Accuracy 0.9693
Epoch 18  Accuracy 0.9702
Epoch 19  Accuracy 0.9708


In [26]:
pred = model(mnist_test_dataset.data / 255.)
is_correct = (torch.argmax(pred, dim=1) == mnist_test_dataset.targets).float()
print(f'Test accuracy: {is_correct.mean():.4f}')

Test accuracy: 0.9583


In [27]:
pred[0]

tensor([2.8743e-24, 1.9608e-28, 9.4058e-19, 1.0751e-12, 5.0737e-25, 1.7463e-23,
        0.0000e+00, 1.0000e+00, 9.5882e-19, 4.4985e-20],
       grad_fn=<SelectBackward0>)

In [34]:
assert sum(pred[0]) == 1